# Irvin EDA
Irvin's notebook with EDA and data processing.

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, r2_score
import plotly.express as px

In [33]:
SEED = 42

## Combine Well Data

Combining all the well data, properly adding NaNs, and adding positional information to the WELL_MASTER.csv

In [3]:
well_log_dir = r"/work/Carpe-Datum-2022/Well_Log/"

In [92]:
well_positional_df = pd.read_csv("/work/Carpe-Datum-2022/Well_Head_and_Completion.csv").set_index("WELL")

In [93]:
def process_data(filename: str) -> pd.DataFrame:
    """
    Combines the well petro/geo-physical information and adds the positional data for each well.
    """
    df = pd.read_csv(well_log_dir + filename)
    well_id = filename.split(".")[0]
    
    df["well_id"] = well_id
    df["X"] = well_positional_df.loc[well_id, "X(ft)"]
    df["Y"] = well_positional_df.loc[well_id, "Y(ft)"]
    df["Completion Zone"] = well_positional_df.loc[well_id, "Completion"]
    return df

In [94]:
# Grab all WP files
files = os.listdir(well_log_dir)
master = pd.DataFrame()

# Read in files into a list of dataframes, excluding ourselves
wells = [process_data(file) for file in files if "WELL_MASTER" not in file]

master = pd.concat(wells)

In [95]:
numerical_columns = master.dtypes[master.dtypes != "object"].index
numerical_columns

Index(['MD(ft)', 'PORO(v/v)', 'Permeability(mD)', 'RHOB(g/cm3)', 'DTS(us/ft)',
       'DT(us/ft)', 'PEF(B/E)', 'RD(OHMM)', 'RS(OHMM)', 'ROP', 'DENC(g/cm3)',
       'NPHI(v/v)', 'X', 'Y'],
      dtype='object')

In [104]:
# Hard-coded because I'm too lazy to figure out how to do this correctly.
master[numerical_columns] = np.where(master[numerical_columns] > -300,  master[numerical_columns], np.NaN)
master = master.replace(-999, np.NaN)
master.describe()

,MD(ft),PORO(v/v),Permeability(mD),RHOB(g/cm3),DTS(us/ft),DT(us/ft),PEF(B/E),RD(OHMM),RS(OHMM),ROP,DENC(g/cm3),NPHI(v/v),X,Y
count,1963.000000,455.000000,221.000000,1918.000000,1001.000000,1920.000000,1689.000000,1724.000000,1724.000000,1721.000000,1684.000000,1963.000000,1963.000000,1963.000000
mean,4673.224657,0.131140,10.497034,2.472862,129.890987,80.322722,5.351575,181.844254,514.513168,21.641036,0.047522,0.200203,124875.694213,87554.249042
std,165.718326,0.062887,39.248645,0.207148,18.915431,17.827921,2.475290,184.443838,3246.082233,7.711698,0.027440,0.107541,9796.471857,3209.167562
min,4228.005683,0.019566,0.000001,1.991035,75.056660,1.138524,0.019155,10.197411,2.270108,2.760923,-0.050168,-0.001900,109560.260000,81747.120000
25%,4546.547858,0.072468,0.019500,2.326909,116.847254,69.493418,4.930555,70.345749,71.606283,16.840993,0.030830,0.122878,115987.290000,84476.470000
50%,4666.802978,0.129662,0.094813,2.469486,131.961625,78.929504,5.782597,130.884864,137.398654,20.014150,0.050912,0.186431,124094.800000,88549.740000
75%,4796.413228,0.185373,1.948104,2.578890,142.296407,87.863830,6.477832,213.021581,259.946834,27.341605,0.063165,0.244933,134042.860000,89987.000000
max,5096.702794,0.249241,352.697773,3.044163,186.253964,179.133001,11.490936,2508.770579,62290.800000,46.895400,0.161357,1.021738,142095.820000,93691.550000


In [105]:
ai = pd.read_csv('/work/Carpe-Datum-2022/AI.csv')

In [106]:
master.groupby("well_id").mean(["PORO(v/v)", "Permeability(md)"])

,MD(ft),PORO(v/v),Permeability(mD),RHOB(g/cm3),DTS(us/ft),DT(us/ft),PEF(B/E),RD(OHMM),RS(OHMM),ROP,DENC(g/cm3),NPHI(v/v),X,Y
well_id,,,,,,,,,,,,,,
WP0,4725.391739,0.128137,NaN,2.570550,130.203580,75.877758,7.374498,NaN,NaN,15.932319,0.061904,0.209666,137106.82,83818.47
WP1,4868.900545,NaN,NaN,2.469263,NaN,92.942081,NaN,176.966978,175.689880,NaN,NaN,0.279190,132460.98,85832.77
WP10,4818.174657,0.123507,3.681348,2.392931,NaN,86.678756,4.433433,196.608451,188.396267,18.849304,0.017321,0.213481,134042.86,82411.27
WP11,4661.109023,0.099223,0.453895,2.544808,110.119546,67.390233,7.044380,233.012950,329.287366,15.102250,0.052180,0.134440,115725.49,86866.60
WP12,4808.270995,NaN,NaN,2.470077,143.509402,79.574038,0.042712,133.065740,213.803551,26.269538,0.042712,0.201439,133136.98,93691.55
WP13,4531.991055,NaN,NaN,2.470225,135.242368,79.086781,6.662322,170.883516,277.129193,15.320779,0.059148,0.192889,119311.27,89234.02
WP14,4781.602978,0.138346,NaN,2.476446,NaN,75.289129,5.762926,144.473656,173.289089,19.841724,0.058958,0.154997,134837.26,83865.82
WP15,4405.841183,0.172151,NaN,2.342374,NaN,94.616725,NaN,82.461831,83.107373,NaN,NaN,0.340167,109560.26,86833.62
WP16,4562.928038,0.144393,NaN,2.482244,NaN,76.430410,5.907931,145.416383,169.218916,19.972302,0.059421,0.160589,112898.58,91601.46


In [107]:
master.to_csv(well_log_dir + "WELL_MASTER.csv", index=False)

In [123]:
prod_df = pd.read_csv("/work/Carpe-Datum-2022/Production_History_Field.csv", index_col="Date")

In [11]:
# prod_df["Total"] = prod_df.sum(axis=1)

In [12]:
prod_df.head()

,WOPR:WP0(bbl/day),WOPR:WP1(bbl/day),WOPR:WP10(bbl/day),WOPR:WP11(bbl/day),WOPR:WP12(bbl/day),WOPR:WP13(bbl/day),WOPR:WP14(bbl/day),WOPR:WP15(bbl/day),WOPR:WP16(bbl/day),WOPR:WP17(bbl/day),...,WOPR:WP46(bbl/day),WOPR:WP47(bbl/day),WOPR:WP48(bbl/day),WOPR:WP49(bbl/day),WOPR:WP5(bbl/day),WOPR:WP6(bbl/day),WOPR:WP7(bbl/day),WOPR:WP8(bbl/day),WOPR:WP9(bbl/day),Total
Date,,,,,,,,,,,,,,,,,,,,,
1-Jan-12,8.945508,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,381.691952
1-Feb-12,8.854879,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,344.603663
1-Mar-12,8.845839,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,340.549643
1-Apr-12,8.839578,0.0,0.0,0,0.0,0.0,0.0,3483.552017,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3821.070871
1-May-12,8.834659,0.0,0.0,0,0.0,0.0,0.0,3374.525497,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.598606,3717.077014


## Production Plot

Plot of time series data for each well's production, as well as the total production for the collection, as a whole.

In [126]:
px.line(prod_df, x=prod_df.index, y=prod_df.columns)

In [205]:
prod_df.index = pd.to_datetime(prod_df.index)
prod_df

,WP0,WP1,WP10,WP11,WP12,WP13,WP14,WP15,WP16,WP17,...,WP45,WP46,WP47,WP48,WP49,WP5,WP6,WP7,WP8,WP9
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-01,8.945508,0.0,0.000000,0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2012-02-01,8.854879,0.0,0.000000,0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2012-03-01,8.845839,0.0,0.000000,0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2012-04-01,8.839578,0.0,0.000000,0,0.0,0.000000,0.0,3483.552017,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2012-05-01,8.834659,0.0,0.000000,0,0.0,0.000000,0.0,3374.525497,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,7.598606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-01,0.000000,0.0,45.614398,0,0.0,21.792120,0.0,0.000000,0.0,0.0,...,0.141282,1158.309413,1069.630665,279.027042,227.041588,7.599481,738.835288,5.635710,0.0,0.000000
2021-10-01,0.000000,0.0,45.574879,0,0.0,21.835644,0.0,0.000000,0.0,0.0,...,0.141266,1146.518933,1058.876023,273.061398,222.216328,7.437834,724.097063,5.502796,0.0,0.000000
2021-11-01,0.000000,0.0,45.537784,0,0.0,21.877102,0.0,0.000000,0.0,0.0,...,0.141252,1135.281660,1048.622946,267.409676,217.636459,7.284528,710.403922,5.383982,0.0,0.000000


In [214]:
start_dates = prod_df.apply(lambda x: x.index[x.diff().argmax()]).rename("Start")
start_dates[["WP0", "WP18"]] = prod_df.index[0]
start_dates = pd.to_datetime(start_dates) - pd.DateOffset(months=1)

In [215]:
distance_scoring_input = pd.concat([start_dates, well_positional_df[["X(ft)","Y(ft)"]]], axis=1)
distance_scoring_input

,Start,X(ft),Y(ft)
WP0,2011-12-01,137106.82,83818.47
WP1,2016-10-01,132460.98,85832.77
WP10,2019-03-01,134042.86,82411.27
WP11,2012-01-01,115725.49,86866.60
WP12,2013-11-01,133136.98,93691.55
WP13,2015-05-01,119311.27,89234.02
WP14,2014-06-01,134837.26,83865.82
WP15,2012-03-01,109560.26,86833.62
WP16,2017-11-01,112898.58,91601.46
WP17,2012-07-01,112766.46,88549.74


In [216]:
cumsum_matrix.index = pd.to_datetime(cumsum_matrix.index)

In [201]:
distance_scoring_input["X(ft)"]

WP0     137106.82
WP1     132460.98
WP10    134042.86
WP11    115725.49
WP12    133136.98
WP13    119311.27
WP14    134837.26
WP15    109560.26
WP16    112898.58
WP17    112766.46
WP18    117798.78
WP19    110189.22
WP2     133634.00
WP20    133687.47
WP21    121120.49
WP22    115987.29
WP23    139326.01
WP24    142095.82
WP25    128136.21
WP26    118170.11
WP27    129829.39
WP28    120522.78
WP29    138541.42
WP3     140892.57
WP30    131123.52
WP31    136285.14
WP32    128675.17
WP33    123960.21
WP34    134053.71
WP35    122230.81
WP36    111749.78
WP37    134902.25
WP38    116803.31
WP39    111890.88
WP4     109951.43
WP40    126601.30
WP41    114333.01
WP42    136532.27
WP43    115684.27
WP44    118629.90
WP45    129176.30
WP46    139046.81
WP47    137407.15
WP48    124094.80
WP49    123188.65
WP5     123950.95
WP6     111035.32
WP7     112439.69
WP8     127369.74
WP9     115943.69
Name: X(ft), dtype: float64

In [236]:
distance_scoring_input["Scores"] = distance_scoring_input.apply(lambda x: dis_func(x["X(ft)"], x["Y(ft)"], 1, x["Start"]), axis=1)
distance_output = pd.concat([production_df, distance_scoring_input], axis=1)

In [252]:
# Check Rho vs. R^2
rho_choice = [x / 10 for x in range(10, 21)]

for rho in rho_choice:
    scores = distance_scoring_input.apply(lambda x: dis_func(x["X(ft)"], x["Y(ft)"], rho, x["Start"]), axis=1)
    model = LinearRegression()
    true = production_df.values.reshape(-1, 1)
    model.fit(scores.values.reshape(-1, 1), true)
    preds = model.predict(scores.values.reshape(-1, 1))
    score = r2_score(true, preds)
    print(f"For rho={rho}, the model got a linear R^2 of {score}")

distance_output["Scores"] = distance_scoring_input.apply(lambda x: dis_func(x["X(ft)"], x["Y(ft)"], 5, x["Start"]), axis=1)

For rho=1.0, the model got a linear R^2 of 0.012181010562580585
For rho=1.1, the model got a linear R^2 of 0.011958437353974505
For rho=1.2, the model got a linear R^2 of 0.01180896461235581
For rho=1.3, the model got a linear R^2 of 0.011734176907322813
For rho=1.4, the model got a linear R^2 of 0.011734014473425591
For rho=1.5, the model got a linear R^2 of 0.011806837167292028
For rho=1.6, the model got a linear R^2 of 0.011949631077063483
For rho=1.7, the model got a linear R^2 of 0.012158266163441955
For rho=1.8, the model got a linear R^2 of 0.012427743004579161
For rho=1.9, the model got a linear R^2 of 0.0127523997365806
For rho=2.0, the model got a linear R^2 of 0.013126074827247591


In [253]:
px.scatter(distance_output, x="Scores", y="sum", hover_name=distance_output.index)

In [255]:

distance_output["Scores"] = distance_scoring_input.apply(lambda x: dis_func(x["X(ft)"], x["Y(ft)"], 2, x["Start"]), axis=1)
px.scatter(distance_output, x="Scores", y="sum", hover_name=distance_output.index)


In [218]:
prod_df.columns = [x.replace("WOPR:", "").replace("(bbl/day)","") for x in prod_df.columns]
prod_df

,WP0,WP1,WP10,WP11,WP12,WP13,WP14,WP15,WP16,WP17,...,WP45,WP46,WP47,WP48,WP49,WP5,WP6,WP7,WP8,WP9
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-01,8.945508,0.0,0.000000,0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2012-02-01,8.854879,0.0,0.000000,0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2012-03-01,8.845839,0.0,0.000000,0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2012-04-01,8.839578,0.0,0.000000,0,0.0,0.000000,0.0,3483.552017,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2012-05-01,8.834659,0.0,0.000000,0,0.0,0.000000,0.0,3374.525497,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,7.598606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-01,0.000000,0.0,45.614398,0,0.0,21.792120,0.0,0.000000,0.0,0.0,...,0.141282,1158.309413,1069.630665,279.027042,227.041588,7.599481,738.835288,5.635710,0.0,0.000000
2021-10-01,0.000000,0.0,45.574879,0,0.0,21.835644,0.0,0.000000,0.0,0.0,...,0.141266,1146.518933,1058.876023,273.061398,222.216328,7.437834,724.097063,5.502796,0.0,0.000000
2021-11-01,0.000000,0.0,45.537784,0,0.0,21.877102,0.0,0.000000,0.0,0.0,...,0.141252,1135.281660,1048.622946,267.409676,217.636459,7.284528,710.403922,5.383982,0.0,0.000000


In [211]:
date_well_interaction_matrix = pd.concat([prod_df.cumsum(axis=0).loc[start_dates[0]], well_positional_df[["X(ft)", "Y(ft)"]]], axis=1)
date_well_interaction_matrix

,2012-01-01 00:00:00,X(ft),Y(ft)
WP0,8.945508,137106.82,83818.47
WP1,0.000000,132460.98,85832.77
WP10,0.000000,134042.86,82411.27
WP11,0.000000,115725.49,86866.60
WP12,0.000000,133136.98,93691.55
WP13,0.000000,119311.27,89234.02
WP14,0.000000,134837.26,83865.82
WP15,0.000000,109560.26,86833.62
WP16,0.000000,112898.58,91601.46
WP17,0.000000,112766.46,88549.74


In [245]:
cumsum_matrix = prod_df.cumsum(axis=0)

def dis_func(x: float,
             y: float,
             rho: float,
             date: str) -> float:

    if date not in cumsum_matrix.index:
        return 0
    df = pd.concat([cumsum_matrix.loc[date], well_positional_df[["X(ft)", "Y(ft)"]]], axis=1)

    df["Distance"] = df.apply(lambda row: ((row["X(ft)"] - x) ** 2 + (row["Y(ft)"] - y) ** 2) ** (1/2), axis=1)
    df["Score"] = df[date] / (df["Distance"]) ** rho

    return df["Score"].sum()

In [161]:
dis_func(13000, 90000, .1, start_dates[0])

Here
There


0.0030340646972473705

In [14]:
well_time_series = [prod_df[prod_df[column] != 0][column].reset_index(drop=True) for column in prod_df.columns]

In [133]:
prod_df

,WOPR:WP0(bbl/day),WOPR:WP1(bbl/day),WOPR:WP10(bbl/day),WOPR:WP11(bbl/day),WOPR:WP12(bbl/day),WOPR:WP13(bbl/day),WOPR:WP14(bbl/day),WOPR:WP15(bbl/day),WOPR:WP16(bbl/day),WOPR:WP17(bbl/day),...,WOPR:WP45(bbl/day),WOPR:WP46(bbl/day),WOPR:WP47(bbl/day),WOPR:WP48(bbl/day),WOPR:WP49(bbl/day),WOPR:WP5(bbl/day),WOPR:WP6(bbl/day),WOPR:WP7(bbl/day),WOPR:WP8(bbl/day),WOPR:WP9(bbl/day)
Date,,,,,,,,,,,,,,,,,,,,,
1-Jan-12,8.945508,0.0,0.000000,0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1-Feb-12,8.854879,0.0,0.000000,0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1-Mar-12,8.845839,0.0,0.000000,0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1-Apr-12,8.839578,0.0,0.000000,0,0.0,0.000000,0.0,3483.552017,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1-May-12,8.834659,0.0,0.000000,0,0.0,0.000000,0.0,3374.525497,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,7.598606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1-Sep-21,0.000000,0.0,45.614398,0,0.0,21.792120,0.0,0.000000,0.0,0.0,...,0.141282,1158.309413,1069.630665,279.027042,227.041588,7.599481,738.835288,5.635710,0.0,0.000000
1-Oct-21,0.000000,0.0,45.574879,0,0.0,21.835644,0.0,0.000000,0.0,0.0,...,0.141266,1146.518933,1058.876023,273.061398,222.216328,7.437834,724.097063,5.502796,0.0,0.000000
1-Nov-21,0.000000,0.0,45.537784,0,0.0,21.877102,0.0,0.000000,0.0,0.0,...,0.141252,1135.281660,1048.622946,267.409676,217.636459,7.284528,710.403922,5.383982,0.0,0.000000


In [ ]:
well_time_series_df

## Decline Curve Analysis DataFrame

Has all 0 values removed, so that the initial, as well as the 

In [43]:
well_time_series_df = pd.concat(well_time_series, axis=1)
well_time_series_df.describe()

,WOPR:WP0(bbl/day),WOPR:WP1(bbl/day),WOPR:WP10(bbl/day),WOPR:WP11(bbl/day),WOPR:WP12(bbl/day),WOPR:WP13(bbl/day),WOPR:WP14(bbl/day),WOPR:WP15(bbl/day),WOPR:WP16(bbl/day),WOPR:WP17(bbl/day),...,WOPR:WP46(bbl/day),WOPR:WP47(bbl/day),WOPR:WP48(bbl/day),WOPR:WP49(bbl/day),WOPR:WP5(bbl/day),WOPR:WP6(bbl/day),WOPR:WP7(bbl/day),WOPR:WP8(bbl/day),WOPR:WP9(bbl/day),Total
count,54.000000,26.000000,34.000000,0.0,87.000000,80.000000,46.000000,15.000000,44.000000,13.000000,...,22.000000,30.000000,63.000000,54.000000,77.000000,30.000000,12.000000,50.000000,41.000000,121.000000
mean,8.809168,38.675641,46.363101,NaN,0.664632,22.342583,381.194628,3060.205978,1221.216449,1367.727037,...,1249.728244,1215.319622,464.882440,351.281916,13.723622,1033.318772,5.925703,13.095474,6.434229,8168.554806
std,0.024437,0.271776,0.675604,NaN,0.044145,0.867709,45.894232,223.213725,388.403162,73.441095,...,94.585474,131.419452,127.722299,89.236190,3.947069,273.942482,0.544118,0.098262,0.674547,2647.876209
min,8.785622,38.346559,45.500584,NaN,0.574007,21.417261,297.860478,2744.731762,650.708425,1263.301841,...,1123.836187,1038.169784,261.701444,213.002484,7.129533,696.642534,5.269145,12.969189,5.498970,340.549643
25%,8.795126,38.467457,45.795829,NaN,0.629549,21.613251,343.387504,2890.105172,869.892989,1305.778714,...,1172.948476,1105.357913,353.050039,274.153564,10.257872,799.168559,5.473093,13.017598,5.865219,6632.304585
50%,8.804288,38.611888,46.258112,NaN,0.674357,21.998420,390.596558,3030.348207,1199.865209,1375.149992,...,1239.033637,1189.925342,464.081421,348.436063,13.952722,979.260694,5.855899,13.075749,6.280917,8075.815412
75%,8.814843,38.818828,46.803097,NaN,0.707594,22.946195,420.371976,3215.375697,1549.219310,1421.757508,...,1311.294321,1335.117225,578.041027,426.108004,17.214352,1226.342823,6.320583,13.152950,7.059284,9800.313135
max,8.945508,39.393796,48.109917,NaN,0.717781,24.529675,447.046761,3483.552017,1941.703124,1484.466481,...,1441.033779,1454.967513,676.068447,512.678527,19.817210,1611.393460,6.854249,13.385876,7.598606,13674.068207


In [115]:
two_year_well_df = well_time_series_df.loc[:, well_time_series_df.isnull().sum() < (121-24)].T
two_year_all = well_time_series_df.T.loc[:, 0:24].sum(axis=1)
two_year_missing_df = well_time_series_df.loc[:, well_time_series_df.isnull().sum() > (121-24)].T.drop("WOPR:WP11(bbl/day)")
two_year_missing_df

,0,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,120
WOPR:WP15(bbl/day),3483.552017,3374.525497,3303.352197,3245.098305,3185.653088,3130.273538,3077.372499,3030.348207,2985.738710,2944.552482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOPR:WP17(bbl/day),1484.466481,1459.745523,1439.459329,1421.757508,1404.787529,1388.843070,1375.149992,1352.293453,1329.025359,1305.778714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOPR:WP20(bbl/day),40.382349,40.158583,40.026658,39.923570,39.838491,39.767086,39.701549,39.643947,39.589288,39.538570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOPR:WP25(bbl/day),1405.446092,1384.253383,1370.772089,1358.965948,1348.232943,1338.514013,1328.952458,1320.055693,1311.161822,1302.513261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOPR:WP31(bbl/day),510.620476,504.529870,500.399755,496.636706,491.478856,485.768756,480.231606,475.136240,470.136999,465.512308,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOPR:WP32(bbl/day),1618.477384,1618.477384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOPR:WP35(bbl/day),2.219789,2.212678,2.208465,2.204942,2.201791,2.198944,2.196150,2.193553,2.190964,2.190964,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOPR:WP39(bbl/day),13.695334,13.527959,13.440102,13.384069,13.340832,13.305672,13.278049,13.250760,13.226486,13.203026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOPR:WP40(bbl/day),49.518124,49.518124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOPR:WP46(bbl/day),1441.033779,1407.980521,1382.227311,1358.252662,1335.846990,1315.231578,1299.482550,1285.634109,1271.689996,1257.950561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
two_year_missing_df.columns = [str(x) for x in two_year_missing_df.columns]
two_year_missing_df["decline_init"] = two_year_missing_df["1"] - two_year_missing_df["0"]


In [261]:
two_year_well_df.columns = [str(x) for x in two_year_well_df.columns]
two_year_well_df
# two_year_well_df = two_year_well_df.drop("Total", axis=0)
two_year_well_df["decline_init"] = two_year_well_df["1"] - two_year_well_df["0"]
two_year_well_df["sum"] = two_year_well_df.loc[:, "0":"24"].sum(axis=1)
two_year_X = two_year_well_df[["0",]]
two_year_y = two_year_well_df["sum"]
train_X, test_X, train_y, test_y = train_test_split(two_year_X, two_year_y, test_size=.2, random_state=SEED)
px.scatter(two_year_well_df, x="0", y="sum", hover_name=two_year_well_df.index)

In [259]:
two_year_model = LinearRegression()

two_year_model.fit(train_X, train_y)
two_year_preds = two_year_model.predict(test_X)
two_year_missing_df["sum"] = two_year_model.predict(two_year_missing_df[["0"]])

In [121]:
pd.concat([two_year_missing_df["sum"], two_year_well_df["sum"]]).to_csv("/work/Carpe-Datum-2022/EDA_notebooks/two_year_forecast.csv")

In [224]:
production_df = pd.concat([two_year_missing_df["sum"], two_year_well_df["sum"]])
production_df.index = [well.replace("WOPR:","").replace("(bbl/day)","") for well in production_df.index]
production_df

WP15      89215.309574
WP17      23612.440193
WP20       3339.782867
WP25      20873.420692
WP31       8369.262930
WP32       5385.256943
WP35       3124.896853
WP39       3260.366578
WP40       3185.902346
WP46      29758.420675
WP7        3282.448588
WP0         220.623568
WP1         967.220104
WP10       1165.743198
WP12         17.758218
WP13        586.702696
WP14      10456.887672
WP16      37745.757428
WP18       7935.172962
WP19        178.579386
WP2         134.496098
WP21         39.829815
WP22        328.990467
WP23        159.580914
WP24       2604.450074
WP26      12886.843013
WP27        228.263249
WP28       3505.962752
WP29      22318.279395
WP3       44507.628195
WP30         28.281087
WP33          5.737569
WP34      37084.261716
WP36      10455.426166
WP37      12317.922407
WP38      30098.408009
WP4        7103.245567
WP41      26849.186859
WP42      33045.491963
WP43      77690.934308
WP44      14361.013330
WP45          3.588712
WP47      31206.119465
WP48      1

In [260]:
error = mean_absolute_percentage_error(two_year_preds, test_y)
r2_scored = r2_score(two_year_preds, test_y)
r2_scored

0.9373066813133754

In [ ]:
# I don't know what I'm trying to do here and the math is obviously wrong too.
well_mean_decline_factor = {well.name: (1 - (well.iloc[0] - well.iloc[-1]) ** (1/ well.shape[0])) for well in well_time_series if well.get(0)}
well_mean_decline_factor

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning:

invalid value encountered in double_scalars



{'WOPR:WP0(bbl/day)': 0.03338011693272547,
 'WOPR:WP1(bbl/day)': -0.0017768093629044746,
 'WOPR:WP10(bbl/day)': -0.02861029665315029,
 'WOPR:WP12(bbl/day)': 0.02204661025897181,
 'WOPR:WP13(bbl/day)': -0.012068401578415822,
 'WOPR:WP14(bbl/day)': -0.11494893222843094,
 'WOPR:WP15(bbl/day)': -0.5532305342342627,
 'WOPR:WP16(bbl/day)': -0.17680045764116303,
 'WOPR:WP17(bbl/day)': -0.5148259825319825,
 'WOPR:WP18(bbl/day)': -0.05070721267759781,
 'WOPR:WP19(bbl/day)': 0.02663099846443351,
 'WOPR:WP2(bbl/day)': 0.011567335031054893,
 'WOPR:WP20(bbl/day)': -0.01031891904055593,
 'WOPR:WP21(bbl/day)': 0.06256979212629499,
 'WOPR:WP22(bbl/day)': 0.05424319710998837,
 'WOPR:WP23(bbl/day)': 0.013197189081264882,
 'WOPR:WP24(bbl/day)': -0.0766858558029837,
 'WOPR:WP25(bbl/day)': -0.4197542441673068,
 'WOPR:WP26(bbl/day)': -0.1555376181812389,
 'WOPR:WP27(bbl/day)': 0.01775273875839123,
 'WOPR:WP28(bbl/day)': -0.046417321467089545,
 'WOPR:WP29(bbl/day)': -0.1889305641597574,
 'WOPR:WP3(bbl/day)':

## Average Production relative to position

In [ ]:
well_time_series_df.columns = [name[5:-9] for name in well_time_series_df.columns]

In [ ]:
avg_prod_df = well_time_series_df.mean().rename("Average_Production")
avg_prod_df

WP0        8.809168
WP1       38.675641
WP10      46.363101
WP11            NaN
WP12       0.664632
WP13      22.342583
WP14     381.194628
WP15    3060.205978
WP16    1221.216449
WP17    1367.727037
WP18     291.086371
WP19       7.082097
WP2        5.276535
WP20      39.545986
WP21       1.579448
WP22      13.159516
WP23       6.300324
WP24      90.374282
WP25    1328.452893
WP26     439.336342
WP27       9.066930
WP28     131.235255
WP29     859.540421
WP3     1606.279426
WP30       1.125125
WP31     460.674842
WP32    1618.477384
WP33       0.225082
WP34    1475.869991
WP35       2.201824
WP36     376.967843
WP37     377.621596
WP38     932.002925
WP39      13.290233
WP4      257.113535
WP40      49.518124
WP41    1056.826177
WP42    1146.619302
WP43    2175.493437
WP44     444.719213
WP45       0.142503
WP46    1249.728244
WP47    1215.319622
WP48     464.882440
WP49     351.281916
WP5       13.723622
WP6     1033.318772
WP7        5.925703
WP8       13.095474
WP9        6.434229


In [ ]:
sum_prod_df = well_time_series_df.sum().rename("Sum_Production")
new_df = pd.concat([well_positional_df, sum_prod_df], axis=1).dropna(subset=["X(ft)"], axis=0)
new_df.to_csv('new_df2.csv')

In [ ]:
new_df = pd.concat([well_positional_df, avg_prod_df], axis=1).dropna(subset=["X(ft)"], axis=0)
new_df

,X(ft),Y(ft),Completion,Elevation Kelly Bushing (ft),Average_Production
WP0,137106.82,83818.47,Upper,193.32,8.809168
WP1,132460.98,85832.77,Upper,187.15,38.675641
WP2,133634.00,82144.52,Upper,164.55,5.276535
WP3,140892.57,81747.12,Lower,179.92,1606.279426
WP4,109951.43,89343.45,Lower,186.21,257.113535
WP5,123950.95,92227.31,Lower,172.80,13.723622
WP6,111035.32,88692.32,Lower,182.66,1033.318772
WP7,112439.69,85560.85,Lower,181.09,5.925703
WP8,127369.74,91739.99,Upper,180.04,13.095474
WP9,115943.69,86730.44,Lower,178.25,6.434229


In [ ]:
px.scatter_3d(new_df, x="X(ft)", y="Y(ft)", z="Sum_Production", color="Sum_Production")

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Surface(z=np.diag(new_df["Average_Production"].values), x=new_df["X(ft)"].values, y=new_df["Y(ft)"].values)])

fig.show()

In [ ]:
np.diag(new_df["Average_Production"].values)

array([[   8.80916814,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,   38.67564086,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    5.27653451, ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [   0.        ,    0.        ,    0.        , ..., 1215.31962223,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
         464.88244033,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,  351.28191581]])

In [ ]:
new_df.to_csv('new_df.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6b71fecc-b405-4b93-bce3-c5ed6c21c449' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>